# Script para la obtención de datos

<div align = "justify">
El presente script esta destinado a la recolección de datos correspondientes al Order Book de diferentes criptomonedas, a su procesamiento y a la exportación de los datos resultantes a ficheros csv. El programa de extracción de datos se basa en los pasos descritos en el enlace [1] y los datos extraidos proceden de [2].
</div>

[1] <br>
https://tiao.io/post/exploring-the-binance-cryptocurrency-exchange-api-orderbook/# <br>

[2] <br> 
https://www.binance.com/en/trade/ETH_BUSD?type=spot

In [1]:
# API 
import requests

# Tratamiento base de datos
import pandas as pd

# Tiempo
from datetime import datetime, date, timedelta
import time

In [13]:
def get_data(depth, crpt):
    
    # depth: profundidad del Order Book por lado
    # crpt: criptomoneda (ETHBUSD, BTCBUSD, ...)
    
    # Momento temporal actual
    tdy = datetime.now()
    # año-mes-dia
    filetdy = tdy.strftime("%y") + tdy.strftime("%m") + tdy.strftime("%d")
    # hora-mes
    filemmn = tdy.strftime("%H") + '_' + tdy.strftime("%M")
    
    # ====================== #
    #  Order Book DataFrame  #
    # ====================== #
    
    # Consulta a Biannce
    ob = requests.get("https://api.binance.com/api/v3/depth", params = dict(symbol = crpt, limit = depth)) 
    obResults = ob.json()
    
    # Transformamos los datos de formato JSON a DataFrame
    frames = {side: pd.DataFrame(data = obResults[side], columns = ["price", "quantity"], dtype = float) 
              for side in ["bids", "asks"]}

    frames_list = [frames[side].assign(side = side) for side in frames]
    obData = pd.concat(frames_list, axis = "index", ignore_index = True, sort = True)
    
    # Momento exacto de la foto del Order Book
    obData['date'] = datetime.now().strftime("%Y-%m-%d, %H:%M:%S")
    
    # Criptomoneda
    obData['crpt'] = crpt
    
    # Escribimos en formato feather
    obData.to_feather('Order_Book/' + crpt + '/' + 'OB_' + crpt + '_' + filetdy + '_' + filemmn + '.ftr')
    
    # ============ #
    #     Price    #
    # ============ #
    
    # El código para la obtención de los precios sería el siguiente. No obstante, tal y cómo se ha indicado en el TFM, por 
    # motivos de asociación temporal entre las instantáneas del Order Book y el precio real de la criptomoneda, se opta por
    # utilizar el mid-price.
    
    # prc = requests.get("https://api.binance.com/api/v3/trades", params = dict(symbol = crpt))
    # prcResults = prc.json()
    
    # prcData = pd.DataFrame(data = prcResults, columns = ['price', 'qty', 'quoteQty', 'time'])
    
    # Convertimos timestamp a la fecha actual (sumamos 2 horas para ajustar zona horaria)
    # prcData['date'] = (prcData['time'] / 1000).astype('datetime64[s]') + timedelta(hours = 2)
    
    # Escribimos en formato feather
    # prcData.to_feather('Prices/' + crpt + '/' + 'PRC_' + crpt + '_' + filetdy + '_' + filemmn + '.ftr')

In [ ]:
# Para bitcoin y ethereum, ejecutamos el programa cada minuto.

starttime = time.time()
while True:
    for cr in ['ETHBUSD', 'BTCBUSD']:
        get_data(5000, cr)    
    time.sleep(60.0 - ((time.time() - starttime) % 60.0))